In [21]:
import pandas as pd
import json


In [22]:
#1st function, reading json file and converting to one dataframe using pandas
def json_to_df(json_file):
    with open(json_file, "r") as f:
        #reading json file
        data = json.loads(f.read())
        
        #flattening json dictionaries into a dataframe
        flatten_payload_1 = pd.json_normalize(data)
        #print(flatten_payload_1)
        
        #looking for whether there is a list value in one of the df columns 
        for i in range(len(flatten_payload_1.iloc[0])):
            
            #if there is a list, we utilize json_normalize for that column 
            if isinstance(flatten_payload_1.iloc[0][i], list):
                column_normalize=flatten_payload_1.columns[i]
                #print(column_normalize)
                
                #splitting column name for record path parameters
                column_normalize_list=column_normalize.split(".")
                #print(column_normalize_list)
                
                #normalizing list of dictionaries data
                column_normalize_data = pd.json_normalize(data, record_path=[[column_normalize_list[0],column_normalize_list[1]]])
                #print(column_normalize_data)
                
                #dropping the column that was normalized, from root df
                flatten_payload_1.drop(column_normalize, inplace=True, axis=1)
                #print(flatten_payload_1)
                
                #concatenating dataframes
                frames = [flatten_payload_1, column_normalize_data]
                final_payload=pd.concat(frames).drop_duplicates().reset_index(drop=True)
                
                #dropping records with all NaN/Null values
                final_payload.dropna(axis=0, how="all", inplace=True)
        
    return final_payload 
                
                

In [23]:
final_payload = json_to_df("take_home_sample.json")

In [27]:
#2nd function to generate csv files for the different interfaces
def df_to_csv(final_payload):
    
    #creating subscriber interface 
    subscriber_cols = [col for col in final_payload.columns if "subscriber" in col]
    subscriber_csv = final_payload.to_csv('subscriber_interface_v2.csv', columns=subscriber_cols, index=False)
    
    #creating contactInformation interface
    #this part is hard-coded, this will be changed once json format is modified
    contactInfo_cols = ['contactType', 'preferenceRank']
    contactInfo_csv = final_payload.to_csv('contactInfo_interface_v2.csv', columns=contactInfo_cols, index=False)
    
    #creating address interface
    address_cols = [col for col in final_payload.columns if "address" in col]
    address_csv = final_payload.to_csv('address_interface_v2.csv', columns=address_cols, index=False)
    
    #creating phoneNumber interface
    phoneNumber_cols = [col for col in final_payload.columns if "phoneNumber" in col]
    phoneNumber_csv = final_payload.to_csv('phoneNumber_interface_v2.csv', columns=phoneNumber_cols, index=False)
    
    return subscriber_csv, contactInfo_csv, address_csv, phoneNumber_csv
    
    

In [28]:
csv_interfaces = df_to_csv(final_payload)